<a href="https://colab.research.google.com/github/kartikgandhi/Cross-Project-Hard-Voting-Edited-Dataset/blob/main/Cross_Project_HV_train_phoenix%2Btest_greenstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [253]:
from numpy import mean
from numpy import std

In [254]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [255]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [256]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [257]:
#file to initalize the weights from
weights = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-phoenix data 4.2-4.3 - phoenix data 4.2-4.3.csv")


In [258]:
weights['Change'] = weights['Change'].map({'yes': 1, 'no': 0})

In [259]:
weights.shape

(1100, 8)

In [260]:
weights.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,Change
0,5,0,16,44,0,0,3,1
1,9,0,23,309,1,70,10,1
2,31,1,17,154,1,58,4,1
3,2,0,16,15,1,33,3,0
4,27,0,36,544,0,78,23,1


In [261]:
X = weights.drop('Change', axis=1)
y = weights['Change']

In [262]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=1)

In [263]:
# get a list of base models
def get_models():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))
	return models

In [264]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores

In [265]:
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)

[0.6831955922865014, 0.6005509641873278, 0.5895316804407713]


In [266]:
data = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-greenstone 3.0.7 - greenstone 3.0.7.csv")

In [267]:
data['change'] = data['change'].map({'YES': 1, 'NO': 0})

In [268]:
data.shape

(601, 8)

In [269]:
data.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,4,0,34,189,1,78,24,1
1,6,0,5,133,1,50,20,1
2,7,0,60,2124,1,0,355,1
3,21,0,15,145,1,85,17,1
4,10,0,13,721,1,91,27,1


In [270]:
X = data.drop('change', axis=1)
y = data['change']

In [271]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [272]:
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='hard', weights=scores)
# fit the ensemble on the training dataset
ensemble=ensemble.fit(X_train, y_train)
# make predictions on test set
yhat = ensemble.predict(X_test)

In [273]:
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Weighted Avg Accuracy: %.3f' % (score*100))

Weighted Avg Accuracy: 81.818


In [274]:
print(yhat)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]


In [275]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,yhat)
print(cm)

[[98  0]
 [22  1]]


In [276]:
# predict probabilities
# from sklearn.metrics import roc_auc_score
# pred_prob = ensemble.predict_proba(X_test)
# auc_score = roc_auc_score(y_test, pred_prob[:,1])
# print("Area Under Curve=")
# print(auc_score)

In [277]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,yhat)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.1884336657203579


In [278]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, yhat)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5217391304347826


In [279]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, yhat)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.20851441405707477


In [280]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90        98
           1       1.00      0.04      0.08        23

    accuracy                           0.82       121
   macro avg       0.91      0.52      0.49       121
weighted avg       0.85      0.82      0.74       121

